# Neueral Network Model to predict winner

In [147]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
import shutil
from pathlib import Path
from time import strftime
import keras_tuner as kt
from keras_tuner import HyperParameters
from functools import partial
import os
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("mvp_data.csv", index_col=0).reset_index(drop=True)
df = df.fillna(0)

In [3]:
# define points+rebounds+assists variable
df['PRA'] = df['PTS'] + df['TRB'] + df['AST']

### Select variables used for regression

In [83]:
# select variables used for regression (mutual_info from lin_reg model)
cols = ['PRA', 'WS/48', 'player_efficiency_rating', 'offensive_box_plus_minus',
       'value_over_replacement_player', 'wl_pct', 'seed']
#X = df[cols]
#y = df['Share']

#### drop 1982, 1999, 2005, 2006

In [32]:
drop_yrs = [1982, 1999, 2005, 2006]
mask = ~df['Year'].isin(drop_yrs)
df = df[mask]

years = list(range(1980,2024))
yrs = [y for y in years if y not in drop_yrs]
rand_years = random.sample(yrs, 4)
test_year = rand_years[0]
valid_years = rand_years[1:]

df_test = df[df['Year'] == test_year]
df_valid = df[df['Year'].isin(valid_years)]
df_train = df[~df['Year'].isin(rand_years)]
X_test =  df_test[cols]
y_test = df_test['Share']
X_valid =  df_valid[cols]
y_valid = df_valid['Share']
X_train = df_train[cols]
y_train = df_train['Share']


In [33]:
rand_years

[2019, 1980, 2023, 1990]

## MSE and ReLU

#### Test network with one year

In [14]:
######## NO BATCH NORMALIZATION or DROPOUT #########
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
initializer = tf.keras.initializers.LecunNormal(seed=42)
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="selu", kernel_initializer=initializer),
    tf.keras.layers.Dense(50, activation="selu", kernel_initializer=initializer),
    tf.keras.layers.Dense(50, activation="selu", kernel_initializer=initializer),
    tf.keras.layers.Dense(50, activation="selu", kernel_initializer=initializer),
    tf.keras.layers.Dense(1)
])

In [34]:
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
#initializer = tf.keras.initializers.LecunNormal(seed=42)
RegularizedDense = partial(tf.keras.layers.Dense, 
                            activation="relu", kernel_initializer='he_normal')

model = tf.keras.Sequential([
    norm_layer,
    RegularizedDense(50),
    #tf.keras.layers.AlphaDropout(0.1),
    RegularizedDense(50),
    #tf.keras.layers.AlphaDropout(0.1),
    RegularizedDense(50),
    #tf.keras.layers.AlphaDropout(0.1),
    RegularizedDense(50),
    #tf.keras.layers.AlphaDropout(0.1),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [24]:
# create folder to store logs of training
shutil.rmtree("nn_test1", ignore_errors=True)

In [25]:
# define function to generate path of the log subdirectory based on the current date and time
# will allow to view training results via TensorBoard
def get_run_logdir(root_logdir="nn_test1"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir,
                                                profile_batch=(100, 200))

2024-01-23 13:47:36.592197: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-01-23 13:47:36.592220: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-01-23 13:47:36.595924: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


#### Train model using MSE

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=15,
                    validation_data=(X_valid,y_valid),
                    callbacks=[tensorboard_cb]
                    )

Epoch 1/15
11/11 [==============================] - 2s 27ms/step - loss: 0.0632 - root_mean_squared_error: 0.2514 - val_loss: 0.0190 - val_root_mean_squared_error: 0.1377
Epoch 2/15
11/11 [==============================] - 0s 9ms/step - loss: 0.0399 - root_mean_squared_error: 0.1996 - val_loss: 0.0380 - val_root_mean_squared_error: 0.1948
Epoch 3/15
11/11 [==============================] - 0s 8ms/step - loss: 0.0319 - root_mean_squared_error: 0.1785 - val_loss: 0.0234 - val_root_mean_squared_error: 0.1530
Epoch 4/15
11/11 [==============================] - 0s 8ms/step - loss: 0.0278 - root_mean_squared_error: 0.1667 - val_loss: 0.0332 - val_root_mean_squared_error: 0.1822
Epoch 5/15
11/11 [==============================] - 0s 8ms/step - loss: 0.0252 - root_mean_squared_error: 0.1588 - val_loss: 0.0213 - val_root_mean_squared_error: 0.1459
Epoch 6/15
11/11 [==============================] - 0s 9ms/step - loss: 0.0246 - root_mean_squared_error: 0.1568 - val_loss: 0.0292 - val_root_mean_s

2024-01-23 13:49:24.693318: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-01-23 13:49:24.693341: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


11/11 [==============================] - 0s 12ms/step - loss: 0.0188 - root_mean_squared_error: 0.1370 - val_loss: 0.0298 - val_root_mean_squared_error: 0.1726
Epoch 13/15
11/11 [==============================] - 0s 16ms/step - loss: 0.0196 - root_mean_squared_error: 0.1400 - val_loss: 0.0341 - val_root_mean_squared_error: 0.1848
Epoch 14/15
11/11 [==============================] - 0s 14ms/step - loss: 0.0155 - root_mean_squared_error: 0.1247 - val_loss: 0.0342 - val_root_mean_squared_error: 0.1850
Epoch 15/15
11/11 [==============================] - 0s 12ms/step - loss: 0.0165 - root_mean_squared_error: 0.1284 - val_loss: 0.0479 - val_root_mean_squared_error: 0.2189


2024-01-23 13:49:25.623564: I tensorflow/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-01-23 13:49:25.668424: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


#### Predict and see results

In [36]:
pred = model(X_test)

In [37]:
print('R2:', r2_score(y_test, pred))
print('MSE:', mean_squared_error(y_test, pred))

R2: 0.7932586104860645
MSE: 0.020453000091513872


In [38]:
results = pd.DataFrame(y_test)
results['prediction'] = pred
results.index = df_test['Player']
results

,Share,prediction
Player,,
Giannis Antetokounmpo,0.932,0.878855
James Harden,0.768,0.853921
Paul George,0.352,0.123961
Nikola Jokić,0.210,0.346324
Stephen Curry,0.173,0.188719
Damian Lillard,0.068,0.068974
Joel Embiid,0.049,0.213362
Kevin Durant,0.025,0.328872
Kawhi Leonard,0.013,0.041332


In [13]:
%load_ext tensorboard
%tensorboard --logdir=./nn_test1

Reusing TensorBoard on port 6006 (pid 72941), started 4 days, 19:40:56 ago. (Use '!kill 72941' to kill it.)

#### Implement early stopping

In [39]:
# impelement early stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=7,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("relu_mse1",
                                                         save_best_only=True)

run_index = 1
run_logdir = Path() / "relu_mse1" / f"run_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [45]:
tf.random.set_seed(42)
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
#initializer = tf.keras.initializers.LecunNormal(seed=42)
RegularizedDense = partial(tf.keras.layers.Dense, 
                            activation="relu", kernel_initializer='he_normal')

model = tf.keras.Sequential([
    norm_layer,
    RegularizedDense(50),
    #tf.keras.layers.AlphaDropout(0.1),
    RegularizedDense(50),
    #tf.keras.layers.AlphaDropout(0.1),
    RegularizedDense(50),
    #tf.keras.layers.AlphaDropout(0.1),
    RegularizedDense(50),
    #tf.keras.layers.AlphaDropout(0.1),
    tf.keras.layers.Dense(1, activation = "sigmoid")
])

In [46]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)
history = model.fit(X_train, y_train, epochs=40,
                    validation_data=(X_valid,y_valid),
                    callbacks=callbacks
                    )

Epoch 1/40
11/11 [==============================] - 2s 28ms/step - loss: 0.0565 - root_mean_squared_error: 0.2378 - val_loss: 0.0552 - val_root_mean_squared_error: 0.2350
Epoch 2/40
11/11 [==============================] - 0s 10ms/step - loss: 0.0405 - root_mean_squared_error: 0.2012 - val_loss: 0.0434 - val_root_mean_squared_error: 0.2084
Epoch 3/40
11/11 [==============================] - 0s 18ms/step - loss: 0.0332 - root_mean_squared_error: 0.1823 - val_loss: 0.0307 - val_root_mean_squared_error: 0.1753
Epoch 4/40
11/11 [==============================] - 0s 10ms/step - loss: 0.0279 - root_mean_squared_error: 0.1669 - val_loss: 0.0297 - val_root_mean_squared_error: 0.1724
Epoch 5/40
11/11 [==============================] - 0s 9ms/step - loss: 0.0242 - root_mean_squared_error: 0.1557 - val_loss: 0.0288 - val_root_mean_squared_error: 0.1698
Epoch 6/40
 1/11 [=>............................] - ETA: 0s - loss: 0.0244 - root_mean_squared_error: 0.1563

INFO:tensorflow:Assets written to: relu_mse1/assets


INFO:tensorflow:Assets written to: relu_mse1/assets


11/11 [==============================] - 2s 154ms/step - loss: 0.0242 - root_mean_squared_error: 0.1554 - val_loss: 0.0203 - val_root_mean_squared_error: 0.1423
Epoch 7/40
11/11 [==============================] - 0s 9ms/step - loss: 0.0213 - root_mean_squared_error: 0.1460 - val_loss: 0.0247 - val_root_mean_squared_error: 0.1573
Epoch 8/40
11/11 [==============================] - 0s 8ms/step - loss: 0.0208 - root_mean_squared_error: 0.1441 - val_loss: 0.0299 - val_root_mean_squared_error: 0.1730
Epoch 9/40
11/11 [==============================] - 0s 12ms/step - loss: 0.0202 - root_mean_squared_error: 0.1420 - val_loss: 0.0273 - val_root_mean_squared_error: 0.1653
Epoch 10/40
11/11 [==============================] - 0s 13ms/step - loss: 0.0172 - root_mean_squared_error: 0.1310 - val_loss: 0.0489 - val_root_mean_squared_error: 0.2210
Epoch 11/40
11/11 [==============================] - 0s 8ms/step - loss: 0.0164 - root_mean_squared_error: 0.1280 - val_loss: 0.0324 - val_root_mean_squared

In [47]:
pred = model(X_test)

In [48]:
print('R2:', r2_score(y_test, pred))
print('MSE:', mean_squared_error(y_test, pred))

R2: 0.7786135153836946
MSE: 0.021901844622225553


In [49]:
results = pd.DataFrame(y_test)
results['prediction'] = pred
results.index = df_test['Player']
results

,Share,prediction
Player,,
Giannis Antetokounmpo,0.932,0.788757
James Harden,0.768,0.679594
Paul George,0.352,0.085190
Nikola Jokić,0.210,0.334042
Stephen Curry,0.173,0.152172
Damian Lillard,0.068,0.206050
Joel Embiid,0.049,0.172204
Kevin Durant,0.025,0.258893
Kawhi Leonard,0.013,0.133559


#### Find best hyperparameters

In [57]:
# function to fine-tune hyperparameters
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-5, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(norm_layer)
    #model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal"))
    model.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
    model.compile(loss="mse", optimizer=optimizer,
                  metrics=["RootMeanSquaredError"])
    norm_layer.adapt(X_train)

    return model


In [58]:
# create RandomSearch tuner
random_search_tuner = kt.RandomSearch(
    build_model, objective=kt.Objective("val_root_mean_squared_error", direction="min"), max_trials=5, overwrite=True,
    directory="my_mvp", project_name="my_rnd_search", seed=42)
# search for best hps
random_search_tuner.search(X_train, y_train, epochs=10,
                           validation_data=(X_valid,y_valid))

Trial 5 Complete [00h 00m 03s]
val_root_mean_squared_error: 0.30875781178474426

Best val_root_mean_squared_error So Far: 0.14141060411930084
Total elapsed time: 00h 00m 19s


In [59]:
top3_hps = random_search_tuner.get_best_hyperparameters(num_trials=3)
best_hps = top3_hps[0].values
list(best_hps.values())

[8, 37, 0.007902373711581125, 'sgd']

In [60]:
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [54]:
best_model.fit(X_train, y_train, epochs=10)
pred = best_model(X_test)

Epoch 1/10
11/11 [==============================] - 2s 4ms/step - loss: 0.0191 - root_mean_squared_error: 0.1381
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0177 - root_mean_squared_error: 0.1331
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0151 - root_mean_squared_error: 0.1230
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0137 - root_mean_squared_error: 0.1172
Epoch 5/10
11/11 [==============================] - 0s 5ms/step - loss: 0.0136 - root_mean_squared_error: 0.1168
Epoch 6/10
11/11 [==============================] - 0s 5ms/step - loss: 0.0124 - root_mean_squared_error: 0.1113
Epoch 7/10
11/11 [==============================] - 0s 5ms/step - loss: 0.0126 - root_mean_squared_error: 0.1122
Epoch 8/10
11/11 [==============================] - 0s 5ms/step - loss: 0.0111 - root_mean_squared_error: 0.1052
Epoch 9/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0121 - root_mean_squar

In [61]:
print('R2:', r2_score(y_test, pred))
print('MSE:', mean_squared_error(y_test, pred))

R2: 0.6828570040562396
MSE: 0.031375070760194766


In [62]:
results = pd.DataFrame(y_test)
results['prediction'] = pred
results.index = df_test['Player']
results

,Share,prediction
Player,,
Giannis Antetokounmpo,0.932,0.922120
James Harden,0.768,0.893387
Paul George,0.352,0.127400
Nikola Jokić,0.210,0.298920
Stephen Curry,0.173,0.402054
Damian Lillard,0.068,0.034414
Joel Embiid,0.049,0.365963
Kevin Durant,0.025,0.305532
Kawhi Leonard,0.013,0.091919


### For loop to find best hps for every year

#### Define functions to evaluate

In [9]:
def mvp_prediction(actual, prediction):
    if actual == prediction:
        return 'Correct'
    else:
        return 'Wrong'

In [10]:
def evaluate(y_test, pred):
    evals = []
    results = pd.DataFrame(y_test)
    results['prediction'] = pred
    results.index = df_test['Player']
    actual = results['Share'].idxmax()
    prediction = results['prediction'].idxmax()
    evals.append(r2_score(y_test, pred))
    evals.append(mean_squared_error(y_test, pred))
    evals.append(mvp_prediction(actual, prediction))
    return evals, results

#### For loop to hold out every year as test set

In [11]:
drop_yrs = [1982, 1999, 2005, 2006]
mask = ~df['Year'].isin(drop_yrs)
df = df[mask]

In [63]:
# empty dict to store results
evals_by_year = {}

best_hps_year={}

results_by_year = {}

drop_yrs = [1982, 1999, 2005, 2006]
years = list(range(1980,2024))
yrs = [y for y in years if y not in drop_yrs]

for year in yrs:
    df_test = df[df['Year'] == year] # create test dataframe of one year
    years1 = list(range(1980,2024))
    yrs1 = [y for y in years1 if y not in drop_yrs]
    yrs1.remove(year) # remove test year from list
    valid_years = random.sample(yrs1, 3) # generate 3 random years for validation data
    df_valid = df[df['Year'].isin(valid_years)] # create validation dataframe
    df_train = df[~df['Year'].isin([year] + list(valid_years))] # training dataframe with remaining years
    X_test =  df_test[cols]
    y_test = df_test['Share']
    X_valid =  df_valid[cols]
    y_valid = df_valid['Share']
    X_train = df_train[cols]
    y_train = df_train['Share']
    random_search_tuner = kt.RandomSearch(build_model, objective=kt.Objective("val_root_mean_squared_error", direction="min"), 
        max_trials=5, overwrite=True, directory="my_mvp", project_name="my_rnd_search", seed=42)
    random_search_tuner.search(X_train, y_train, epochs=10,
                           validation_data=(X_valid, y_valid))
    best_model = random_search_tuner.get_best_models(num_models=1)[0]
    #best_model = top2_models[0]
    best_model.fit(X_train, y_train, epochs=10)
    pred = best_model.predict(X_test)                       
    eval, results = evaluate(y_test, pred)
    evals_by_year[year] = eval
    results_by_year[year] = results
    best_hps = random_search_tuner.get_best_hyperparameters(num_trials=1)[0].values
    #best_hps = top3_hps[0].values    
    best_hps_year[year] = list(best_hps.values())

Trial 5 Complete [00h 00m 04s]
val_root_mean_squared_error: 0.249373197555542

Best val_root_mean_squared_error So Far: 0.2215961366891861
Total elapsed time: 00h 00m 19s
Epoch 1/10
11/11 [==============================] - 2s 4ms/step - loss: 0.0227 - root_mean_squared_error: 0.1508
Epoch 2/10
11/11 [==============================] - 0s 8ms/step - loss: 0.0248 - root_mean_squared_error: 0.1575
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0247 - root_mean_squared_error: 0.1571
Epoch 4/10
11/11 [==============================] - 0s 5ms/step - loss: 0.0188 - root_mean_squared_error: 0.1370
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0175 - root_mean_squared_error: 0.1321
Epoch 6/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0158 - root_mean_squared_error: 0.1258
Epoch 7/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0178 - root_mean_squared_error: 0.1334
Epoch 8/10
11/11 [====================

In [64]:
nn1_res = pd.DataFrame(list(evals_by_year.values()), index=evals_by_year.keys())
nn1_res.columns = ['R2', 'MSE', 'MVP']
nn1_res.index.name='Idx'
#nn1_res.sort_values(by = ['Idx'], ascending = True)
nn1_res.sort_values(by = ['MVP', 'Idx'], ascending = [False, True])

,R2,MSE,MVP
Idx,,,
1983,-0.112554,0.090376,Wrong
1989,0.590280,0.030378,Wrong
1993,0.799527,0.019242,Wrong
1997,0.852155,0.013961,Wrong
1998,0.623990,0.037919,Wrong
2001,0.396298,0.053555,Wrong
2008,0.573160,0.042863,Wrong
2011,0.259300,0.067425,Wrong
2017,-0.777560,0.180726,Wrong


In [65]:
nn1_res['MVP'].value_counts()

Correct    30
Wrong      10
Name: MVP, dtype: int64

In [66]:
nn1_res.mean(axis=0)

R2     0.540409
MSE    0.038495
dtype: float64

In [67]:
best_hps = pd.DataFrame(list(best_hps_year.values()), index=best_hps_year.keys())
best_hps

,0,1,2,3
1980,4,74,0.008611,adam
1981,8,37,0.007902,sgd
1983,4,74,0.008611,adam
1984,4,74,0.008611,adam
1985,4,74,0.008611,adam
1986,4,74,0.008611,adam
1987,8,37,0.007902,sgd
1988,4,74,0.008611,adam
1989,4,74,0.008611,adam
1990,4,74,0.008611,adam


In [68]:
for c in best_hps.columns:
    print(best_hps[c].value_counts())

4    30
8    10
Name: 0, dtype: int64
74    30
37    10
Name: 1, dtype: int64
0.008611    30
0.007902    10
Name: 2, dtype: int64
adam    30
sgd     10
Name: 3, dtype: int64


#### Use best hyperperameters and relu

In [ ]:
# create folder to store logs of training
shutil.rmtree("nn_test2", ignore_errors=True)

In [ ]:
def train_model(X_train, y_train, X_valid, y_valid):
    tf.random.set_seed(42)
    norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
    model = tf.keras.Sequential([
        norm_layer,
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(1, activation = "sigmoid")
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=8e-3)
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    norm_layer.adapt(X_train)
    history = model.fit(X_train, y_train, epochs=30,
                        validation_data=(X_valid,y_valid),
                        callbacks=callbacks
                        )
    return model

In [138]:
# empty dict to store results
evals_by_year = {}

results_by_year={}

drop_yrs = [1982, 1999, 2005, 2006]
years = list(range(1980,2024))
yrs = [y for y in years if y not in drop_yrs]

random.seed(0)

for year in yrs:
    df_test = df[df['Year'] == year] # create test dataframe of one year
    years1 = list(range(1980,2024))
    yrs1 = [y for y in years1 if y not in drop_yrs]
    yrs1.remove(year) # remove test year from list
    valid_years = random.sample(yrs1, 4) # generate 4 random years for validation data
    df_valid = df[df['Year'].isin(valid_years)] # create validation dataframe
    df_train = df[~df['Year'].isin([year] + list(valid_years))] # training dataframe with remaining years
    X_test =  df_test[cols]
    y_test = df_test['Share']
    X_valid =  df_valid[cols]
    y_valid = df_valid['Share']
    X_train = df_train[cols]
    y_train = df_train['Share']
    tf.random.set_seed(42)
    norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
    model = tf.keras.Sequential([
        norm_layer,
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(1, activation = "sigmoid")
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=8e-3)
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    norm_layer.adapt(X_train)
    history = model.fit(X_train, y_train, epochs=30,
                        validation_data=(X_valid,y_valid),
                        callbacks=callbacks
                        )
    pred = model(X_test)                       
    eval, results = evaluate(y_test, pred)
    evals_by_year[year] = eval
    results_by_year[year] = results

Epoch 1/30
11/11 [==============================] - 6s 33ms/step - loss: 0.1048 - root_mean_squared_error: 0.3238 - val_loss: 0.0528 - val_root_mean_squared_error: 0.2298
Epoch 2/30
11/11 [==============================] - 0s 9ms/step - loss: 0.0414 - root_mean_squared_error: 0.2034 - val_loss: 0.0397 - val_root_mean_squared_error: 0.1993
Epoch 3/30
11/11 [==============================] - 0s 9ms/step - loss: 0.0323 - root_mean_squared_error: 0.1798 - val_loss: 0.0391 - val_root_mean_squared_error: 0.1978
Epoch 4/30
11/11 [==============================] - 0s 8ms/step - loss: 0.0319 - root_mean_squared_error: 0.1786 - val_loss: 0.0467 - val_root_mean_squared_error: 0.2160
Epoch 5/30
11/11 [==============================] - 0s 9ms/step - loss: 0.0314 - root_mean_squared_error: 0.1773 - val_loss: 0.0341 - val_root_mean_squared_error: 0.1846
Epoch 6/30
11/11 [==============================] - 0s 9ms/step - loss: 0.0250 - root_mean_squared_error: 0.1580 - val_loss: 0.0398 - val_root_mean_s

In [ ]:
# test with train function
# empty dict to store results
evals_by_year = {}

results_by_year={}

drop_yrs = [1982, 1999, 2005, 2006]
years = list(range(1980,2024))
yrs = [y for y in years if y not in drop_yrs]

random.seed(0)

for year in yrs:
    df_test = df[df['Year'] == year] # create test dataframe of one year
    years1 = list(range(1980,2024))
    yrs1 = [y for y in years1 if y not in drop_yrs]
    yrs1.remove(year) # remove test year from list
    valid_years = random.sample(yrs1, 4) # generate 4 random years for validation data
    df_valid = df[df['Year'].isin(valid_years)] # create validation dataframe
    df_train = df[~df['Year'].isin([year] + list(valid_years))] # training dataframe with remaining years
    X_test =  df_test[cols]
    y_test = df_test['Share']
    X_valid =  df_valid[cols]
    y_valid = df_valid['Share']
    X_train = df_train[cols]
    y_train = df_train['Share']
    train_model(X_train, y_train, X_valid, y_valid)
    pred = model(X_test)                       
    eval, results = evaluate(y_test, pred)
    evals_by_year[year] = eval
    results_by_year[year] = results


In [139]:
nn3_res = pd.DataFrame(list(evals_by_year.values()), index=evals_by_year.keys())
nn3_res.columns = ['R2', 'MSE', 'MVP']
nn3_res.index.name='Idx'
nn3_res.sort_values(by = ['MVP', 'Idx'], ascending = [False, True])

,R2,MSE,MVP
Idx,,,
1983,-0.087821,0.088366,Wrong
1989,0.793446,0.015315,Wrong
1993,0.717219,0.027142,Wrong
1994,0.653487,0.031883,Wrong
1997,0.920397,0.007517,Wrong
1998,0.726747,0.027557,Wrong
2001,0.438344,0.049825,Wrong
2008,0.550228,0.045166,Wrong
2011,0.308159,0.062978,Wrong


In [140]:
nn3_res['MVP'].value_counts()

Correct    29
Wrong      11
Name: MVP, dtype: int64

In [141]:
nn3_res.mean(axis=0)

R2     0.568657
MSE    0.036751
dtype: float64

In [135]:
results_by_year[2011]

,Share,prediction
Player,,
Derrick Rose,0.977,0.585454
Dwight Howard,0.531,0.204874
LeBron James,0.431,0.777740
Kobe Bryant,0.354,0.126026
Kevin Durant,0.157,0.169855
Dirk Nowitzki,0.093,0.062743
Dwyane Wade,0.020,0.463598
Manu Ginóbili,0.017,0.010671
Amar'e Stoudemire,0.007,0.006265


### Train model with entire data, save to file

In [142]:
def train_model(X_train, y_train, X_valid, y_valid):
    tf.random.set_seed(42)
    norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
    model = tf.keras.Sequential([
        norm_layer,
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(74, activation="relu", kernel_initializer="he_normal"),
        tf.keras.layers.Dense(1, activation = "sigmoid")
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=8e-3)
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    norm_layer.adapt(X_train)
    history = model.fit(X_train, y_train, epochs=30,
                        validation_data=(X_valid,y_valid),
                        callbacks=callbacks
                        )
    return model


In [146]:
num_models = 5
trained_models = []

save_folder = 'NN_models'
os.makedirs(save_folder, exist_ok=True)

for i in range(num_models):
    valid_years = random.sample(yrs, 5)
    df_valid = df[df['Year'].isin(valid_years)]
    df_train = df[~df['Year'].isin(valid_years)]
    X_valid = df_valid[cols]
    y_valid = df_valid['Share']
    X_train = df_train[cols]
    y_train = df_train['Share']

    model = train_model(X_train, y_train, X_valid, y_valid)
    trained_models.append(model)

    model.save(os.path.join(save_folder, f'model_{i + 1}.h5'))

Epoch 1/30
11/11 [==============================] - 3s 36ms/step - loss: 0.0619 - root_mean_squared_error: 0.2487 - val_loss: 0.0390 - val_root_mean_squared_error: 0.1975
Epoch 2/30
11/11 [==============================] - 0s 10ms/step - loss: 0.0376 - root_mean_squared_error: 0.1939 - val_loss: 0.0417 - val_root_mean_squared_error: 0.2042
Epoch 3/30
11/11 [==============================] - 0s 9ms/step - loss: 0.0311 - root_mean_squared_error: 0.1763 - val_loss: 0.0357 - val_root_mean_squared_error: 0.1889
Epoch 4/30
11/11 [==============================] - 0s 11ms/step - loss: 0.0309 - root_mean_squared_error: 0.1759 - val_loss: 0.0355 - val_root_mean_squared_error: 0.1885
Epoch 5/30
11/11 [==============================] - 0s 9ms/step - loss: 0.0270 - root_mean_squared_error: 0.1642 - val_loss: 0.0384 - val_root_mean_squared_error: 0.1960
Epoch 6/30
11/11 [==============================] - 0s 9ms/step - loss: 0.0248 - root_mean_squared_error: 0.1576 - val_loss: 0.0296 - val_root_mean

In [149]:
# save dict of best results
filehandler = open('nn_results_year.pkl', 'wb')
pickle.dump(results_by_year, filehandler)

## MSE and Selu

In [47]:
# function to fine-tune hyperparameters
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    model = tf.keras.Sequential()
    model.add(norm_layer)
    #model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="selu", kernel_initializer="lecun_normal"))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss="mse", optimizer=optimizer,
                  metrics=["RootMeanSquaredError"])
    norm_layer.adapt(X_train)

    return model


In [48]:
# empty dict to store results
evals_by_year = {}

best_hps_year={}

results_by_year = {}

drop_yrs = [1982, 1999, 2005, 2006]
years = list(range(1980,2024))
yrs = [y for y in years if y not in drop_yrs]

for year in yrs:
    df_test = df[df['Year'] == year] # create test dataframe of one year
    years1 = list(range(1980,2024))
    yrs1 = [y for y in years1 if y not in drop_yrs]
    yrs1.remove(year) # remove test year from list
    valid_years = random.sample(yrs1, 3) # generate 3 random years for validation data
    df_valid = df[df['Year'].isin(valid_years)] # create validation dataframe
    df_train = df[~df['Year'].isin([year] + list(valid_years))] # training dataframe with remaining years
    X_test =  df_test[cols]
    y_test = df_test['Share']
    X_valid =  df_valid[cols]
    y_valid = df_valid['Share']
    X_train = df_train[cols]
    y_train = df_train['Share']
    random_search_tuner = kt.RandomSearch(build_model, objective=kt.Objective("val_root_mean_squared_error", direction="min"), 
        max_trials=5, overwrite=True, directory="my_mvp", project_name="my_rnd_search", seed=42)
    random_search_tuner.search(X_train, y_train, epochs=10,
                           validation_data=(X_valid, y_valid))
    best_model = random_search_tuner.get_best_models(num_models=1)[0]
    #best_model = top2_models[0]
    best_model.fit(X_train, y_train, epochs=10)
    pred = best_model.predict(X_test)                       
    eval, results = evaluate(y_test, pred)
    evals_by_year[year] = eval
    results_by_year[year] = results
    best_hps = random_search_tuner.get_best_hyperparameters(num_trials=1)[0].values
    #best_hps = top3_hps[0].values    
    best_hps_year[year] = list(best_hps.values())

Trial 5 Complete [00h 00m 04s]
val_root_mean_squared_error: 0.3771508038043976

Best val_root_mean_squared_error So Far: 0.15399572253227234
Total elapsed time: 00h 00m 23s
Epoch 1/10
11/11 [==============================] - 2s 5ms/step - loss: 0.0435 - root_mean_squared_error: 0.2086
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0405 - root_mean_squared_error: 0.2011
Epoch 3/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0387 - root_mean_squared_error: 0.1968
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0385 - root_mean_squared_error: 0.1961
Epoch 5/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0361 - root_mean_squared_error: 0.1901
Epoch 6/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0364 - root_mean_squared_error: 0.1907
Epoch 7/10
11/11 [==============================] - 0s 4ms/step - loss: 0.0348 - root_mean_squared_error: 0.1865
Epoch 8/10
11/11 [==================

In [49]:
nn2_res = pd.DataFrame(list(evals_by_year.values()), index=evals_by_year.keys())
nn2_res.columns = ['R2', 'MSE', 'MVP']
nn2_res.index.name='Idx'
nn2_res.sort_values(by = ['MVP', 'Idx'], ascending = [False, True])

,R2,MSE,MVP
Idx,,,
1981,-0.062162,0.054651,Wrong
1983,-0.235494,0.100362,Wrong
1984,-0.203138,0.074627,Wrong
1988,0.309105,0.062232,Wrong
1989,0.149290,0.063074,Wrong
1990,0.227498,0.056441,Wrong
1993,0.491265,0.048829,Wrong
1997,0.515250,0.045775,Wrong
2001,-0.662953,0.147523,Wrong


In [50]:
nn2_res['MVP'].value_counts()

Correct    25
Wrong      15
Name: MVP, dtype: int64

In [51]:
nn2_res.mean(axis=0)

R2     0.384072
MSE    0.052925
dtype: float64

In [52]:
best_hps = pd.DataFrame(list(best_hps_year.values()), index=best_hps_year.keys())
best_hps

,0,1,2,3
1980,4,74,0.009051,adam
1981,8,37,0.008547,sgd
1983,4,74,0.009051,adam
1984,8,37,0.008547,sgd
1985,4,74,0.009051,adam
1986,4,74,0.009051,adam
1987,8,37,0.008547,sgd
1988,4,74,0.009051,adam
1989,4,74,0.009051,adam
1990,4,74,0.009051,adam


In [53]:
for c in best_hps.columns:
    print(best_hps[c].value_counts())

4    28
8    11
7     1
Name: 0, dtype: int64
74     28
37     11
100     1
Name: 1, dtype: int64
0.009051    28
0.008547    11
0.001248     1
Name: 2, dtype: int64
adam    28
sgd     12
Name: 3, dtype: int64


#### Test using best hyperparameters

In [152]:
# impelement early stopping
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=7,
                                                     restore_best_weights=True)
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("selu_mse1",
                                                         save_best_only=True)

run_index = 1
run_logdir = Path() / "selu_mse1" / f"run_{run_index:03d}"
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [158]:
# empty dict to store results
evals_by_year = {}

results_by_year={}

drop_yrs = [1982, 1999, 2005, 2006]
years = list(range(1980,2024))
yrs = [y for y in years if y not in drop_yrs]

for year in yrs:
    df_test = df[df['Year'] == year] # create test dataframe of one year
    years1 = list(range(1980,2024))
    yrs1 = [y for y in years1 if y not in drop_yrs]
    yrs1.remove(year) # remove test year from list
    random.seed(10)
    valid_years = random.sample(yrs1, 3) # generate 3 random years for validation data
    df_valid = df[df['Year'].isin(valid_years)] # create validation dataframe
    df_train = df[~df['Year'].isin([year] + list(valid_years))] # training dataframe with remaining years
    X_test =  df_test[cols]
    y_test = df_test['Share']
    X_valid =  df_valid[cols]
    y_valid = df_valid['Share']
    X_train = df_train[cols]
    y_train = df_train['Share']
    tf.random.set_seed(42)
    norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
    #initializer = tf.keras.initializers.LecunNormal(seed=42)
    model = tf.keras.Sequential([
        norm_layer,
        tf.keras.layers.Dense(74, activation="selu", kernel_initializer="lecun_normal"),
        tf.keras.layers.Dense(74, activation="selu", kernel_initializer="lecun_normal"),
        tf.keras.layers.Dense(74, activation="selu", kernel_initializer="lecun_normal"),
        tf.keras.layers.Dense(74, activation="selu", kernel_initializer="lecun_normal"),
        tf.keras.layers.Dense(1)
    ])
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    norm_layer.adapt(X_train)
    history = model.fit(X_train, y_train, epochs=30,
                        validation_data=(X_valid,y_valid),
                        callbacks=callbacks
                        )
    pred = model(X_test)                       
    eval, results = evaluate(y_test, pred)
    evals_by_year[year] = eval
    results_by_year[year] = results

Epoch 1/30
12/12 [==============================] - 2s 22ms/step - loss: 7.4989 - root_mean_squared_error: 2.7384 - val_loss: 1.8480 - val_root_mean_squared_error: 1.3594
Epoch 2/30
12/12 [==============================] - 0s 5ms/step - loss: 0.9792 - root_mean_squared_error: 0.9896 - val_loss: 1.7454 - val_root_mean_squared_error: 1.3211
Epoch 3/30
12/12 [==============================] - 0s 5ms/step - loss: 0.5466 - root_mean_squared_error: 0.7393 - val_loss: 0.4281 - val_root_mean_squared_error: 0.6543
Epoch 4/30
12/12 [==============================] - 0s 9ms/step - loss: 0.3471 - root_mean_squared_error: 0.5891 - val_loss: 0.3688 - val_root_mean_squared_error: 0.6073
Epoch 5/30
12/12 [==============================] - 0s 11ms/step - loss: 0.4834 - root_mean_squared_error: 0.6953 - val_loss: 0.6720 - val_root_mean_squared_error: 0.8198
Epoch 6/30
12/12 [==============================] - 0s 7ms/step - loss: 0.4327 - root_mean_squared_error: 0.6578 - val_loss: 0.4823 - val_root_mean_

In [159]:
nn4_res = pd.DataFrame(list(evals_by_year.values()), index=evals_by_year.keys())
nn4_res.columns = ['R2', 'MSE', 'MVP']
nn4_res.index.name='Idx'
nn4_res.sort_values(by = ['MVP', 'Idx'], ascending = [False, True])


,R2,MSE,MVP
Idx,,,
1983,-0.071500,0.087041,Wrong
1987,-0.808806,0.152358,Wrong
1989,0.645508,0.026283,Wrong
1990,0.074076,0.067651,Wrong
1993,0.409492,0.056678,Wrong
1994,0.640889,0.033042,Wrong
1998,0.736587,0.026564,Wrong
2001,0.500095,0.044347,Wrong
2002,0.084124,0.077104,Wrong


In [160]:
nn4_res['MVP'].value_counts()

Correct    26
Wrong      14
Name: MVP, dtype: int64

In [161]:
nn4_res.mean(axis=0)

R2     0.448627
MSE    0.046691
dtype: float64

In [162]:
results_by_year[2023]

,Share,prediction
Player,,
Joel Embiid,0.915,0.495415
Nikola Jokić,0.674,0.730018
Giannis Antetokounmpo,0.606,0.610472
Jayson Tatum,0.280,0.263288
Shai Gilgeous-Alexander,0.046,-0.049087
Donovan Mitchell,0.030,0.041728
Domantas Sabonis,0.027,0.035072
Luka Dončić,0.010,0.302613
Stephen Curry,0.005,0.128049


## Huber and Relu

In [75]:
pd.set_option('display.max_rows', None)
pd.DataFrame(df.groupby(['Year'])['Share'].nlargest(3))

Share
Year           
1980 0     66.5
     1     14.3
     2      8.6
1981 9     65.8
     10    61.3
     11    41.4
1982 45    73.5
     46    58.8
     47    29.4
1983 70    96.0
     71    48.5
     72    40.6
1984 99    85.8
     100   49.1
     101   40.1
1985 116   97.8
     117   33.8
     118   27.9
1986 139   98.1
     140   52.2
     141   26.3
1987 157   94.0
     158   57.6
     159   34.7
1988 174   83.1
     175   65.9
     176   63.5
1989 191   78.2
     192   70.4
     193   42.6
1990 210   69.1
     211   66.7
     212   61.3
1991 224   92.8
     225   51.8
     226   49.6
1992 244   93.8
     245   58.4
     246   35.1
1993 261   85.2
     262   66.0
     263   57.7
1994 275   88.0
     276   72.3
     277   38.6
1995 293   85.8
     294   57.6
     295   50.7
1996 309   98.6
     310   50.8
     311   31.9
1997 326   85.7
     327   83.2
     328   32.7
1998 346   93.4
     347   72.6
     348   37.2
1999 365   70.1
     366   65.5
     367   62.7
2000 386   99.8
     387   33.7
     388   30.3
2001 402   90.4
     403   56.9
     404   46.6
2002 419   75.7
     420   71.2
     421   55.2
2003 437   80.8
     438   73.2
     439   41.7
2004 450   99.1
     451   58.2
     452   42.5
2005 466   83.9
     467   81.3
     468   27.5
2006 483   73.9
     484   55.0
     485   43.5
2007 494   88.2
     495   78.5
     496   40.4
2008 511   87.3
     512   71.0
     513   53.2
2009 528   96.9
     529   57.7
     530   56.2
2010 541   98.0
     542   49.5
     543   48.7
2011 557   97.7
     558   53.1
     559   43.1
2012 570   88.8
     571   73.5
     572   31.8
2013 585   99.8
     586   63.2
     587   39.3
2014 601   98.6
     602   71.3
     603   34.7
2015 618   92.2
     619   72.0
     620   42.5
2016 630  100.0
     631   48.4
     632   48.2
2017 640   87.9
     641   74.6
     642   49.5
2018 651   95.5
     652   73.1
     653   44.1
2019 664   93.2
     665   76.8
     666   35.2
2020 676   95.2
     677   74.6
     678   36.3
2021 688   96.1
     689   58.0
     690   44.9
2022 705   87.5
     706   70.6
     707   59.5
2023 717   91.5
     718   67.4
     719   60.6

In [95]:
def custom_huber(y_true, y_pred):
    error = y_true - y_pred
    is_large_y = y_true > 35
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error)
    return tf.where(is_large_y, squared_loss, linear_loss)
    

In [ ]:
#save at the end, move to the end
model.save("my_keras_model", save_format="tf")